#### A synchronus code is when the interpreter reads the code line by line and executes the function in the order that we have mentioned.
#### Below is an example of a Synchronus code that executes the tasks in the order that we have mentioned in the main() method. In the code given below the time taken for the function to execute depends on the magnitude of the number passed to it. Like wise there might be I/O bound operations in your application that migh take an unpredictable amount of time to complete. Therefore, in such senarios it is better to use Pythons asyncio module and its associated syntaxes to handle those I/O operations.

In [7]:
def is_prime(x: int):
    """Returns True if x is prime"""

    return not any(x//y == x/y for y in range(x-1, 1, -1))

def highest_prime_below(x):

    print(f"Highest prime below {x}")

    for y in range(x-1, 0, -1):
        if is_prime(y):
            print(f"Highest prime number below {x} is {y}")
            return y
    return None                                     # If no prime number found.

def main():
    #_________Tasks___________#
    highest_prime_below(100000)
    highest_prime_below(10000)
    highest_prime_below(1000)

# Programs entry point.
main()

# As you can see in the output below the answer is returned in the order we executed the functions above

Highest prime below 100000
Highest prime number below 100000 is 99991
Highest prime below 10000
Highest prime number below 10000 is 9973
Highest prime below 1000
Highest prime number below 1000 is 997


# asyncio

#### An async piece of code is not designed to execute the tasks in a predefined order, rather an async program rapidly switches between tasks for a certain amount of time defined by .sleep() method. async methods do not make use of multiple threads, there is only a single thread that switches between tasks. Therefore, it is recomended to use asyncio for I/O bound operations and not CPU bound operations. Never use a Blocking statement like time.sleep in an async corutine.

#### Below is an example of async version code of the code above. A function that can be used asynchronusly is called a corutine. To define a corutine use the keyword 'async' before the def keyword. A corutine must contain an await statement that defines where to pause and switch to another task in the eventloop.

### The event sequence of the code given below is as follows:-

#### 1. We first create an eventloop object (line-35), an event-loop is used to define a set of awaitable tasks that are needed to be executed asynchronously. we use the .run_until_complete() on it and pass a corutine to it (line-36), that corutine contains an awaitable wait function. .run_until_complete() will run all the tasks until they all return a value.

#### 2. The main() function has an awaitable assigned to wait function. A wait function takes an iterable of awaitables, runs them asynchronusly, blocks the program to move beyond this point until the return_when is satisfied. In  our case it is set to All_COMPLETE, other options are FIRST_COMPLETED & FIRST_EXCEPTION.

#### 3. highest_prime_below() is a corutine. That means, tasks for this functions can be made to wait at a certain point and then be resumed after a certain amount of time. Therefore, we need to mention at what point in that function we want to pause it, we do that using 'await' keyword. Every async function must have an await statement.

#### 4. In the highest_prime_below() function, we make the taks await after returning y, at await the tasks sleeps for 0.01 second. Note, it uses async.sleep and not time.sleep. time.sleep is a blocking function i.e. It stops the CPU from doing any sort of work. However, async.sleep makes the task to go to sleep for the mentioned time and picks another task from the event_loop and after the time expires resumes the previous session. its this async.sleep method that really makes this piece of code truly asynchronous.

In [10]:
import time
import asyncio

def is_prime(x: int)-> bool:
    """Returns True if x is prime"""

    return not any(x//y == x/y for y in range(x-1, 1, -1))

# creating a corutine
async def highest_prime_below(x):

    print(f"Highest prime below {x}")

    for y in range(x-1, 0, -1):
        if is_prime(y):
            print(f"Highest prime number below {x} is {y}")
            return y
        await asyncio.sleep(0.01)           # suspention point
    return None                                     # If no prime number found.

# Programs entry point.
async def main():

    #_________This will not work_________#
    # await highest_prime_below(100000)
    # await highest_prime_below(10000)
    # await highest_prime_below(1000)

    #________Use this Insted_____________#
    await asyncio.wait([
        highest_prime_below(100000),
        highest_prime_below(10000),
        highest_prime_below(1000) ], return_when=asyncio.ALL_COMPLETED)

loop = asyncio.get_event_loop()     # Initializing an event loop
loop.run_until_complete(main())     # passing a coroutine to our event loop, and making it run until they complete
loop.close()                        # terminating event loop

# The above code doesn't work on notebook, use console insted.

RuntimeError: This event loop is already running